<a href="https://colab.research.google.com/github/HWAN722/Deep-learning-models/blob/main/Llama3_2_3B_Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
!pip install transformers datasets
!pip install evaluate
!pip install bitsandbytes
!pip install zhconv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.6 MB/s eta 0:00:00
     

In [3]:
from datasets import load_dataset

dataset = load_dataset("weiiiii0622/HW1-Chinese_Extractive_QA")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.json:   0%|          | 0.00/6.55M [00:00<?, ?B/s]

valid.json:   0%|          | 0.00/922k [00:00<?, ?B/s]

test.json:   0%|          | 0.00/467k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quant_config = BitsAndBytesConfig(
    load_in_8bit=True
)
# load
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct",device_map="auto",quantization_config=quant_config)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [6]:
import torch
print(f"Model loaded, memory allocated: {torch.cuda.memory_allocated() / 1e9} GB")

Model loaded, memory allocated: 3.610563072 GB


In [7]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear8bitLt(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear8bitLt(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear8bitLt(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear8bitLt(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((

In [5]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=28,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
)

model = get_peft_model(model, lora_config)
print(model.print_trainable_parameters())

trainable params: 4,587,520 || all params: 3,217,337,344 || trainable%: 0.1426
None


In [7]:
import zhconv
import json

def preprocess_function(examples):
    inputs = [zhconv.convert(ex, 'zh-cn') for ex in examples["question"]]
    # answers = [zhconv.convert(ex["text"], 'zh-cn') for ex in examples["answer"]]
    answers = []
    for ex in examples["answer"]:
        if ex is None or ex.get("text") is None or len(ex["text"]) == 0:
            answers.append("")
        else:
            answers.append(zhconv.convert(ex["text"][0], 'zh-cn'))
    outputs = [f"{inp}{out}" for inp, out in zip(inputs, answers)]
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    model_inputs = tokenizer(inputs, padding="max_length", max_length=128, truncation=True)
    labels = tokenizer(outputs, padding="max_length", max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

print(tokenized_datasets)

Map:   0%|          | 0/21714 [00:00<?, ? examples/s]

Map:   0%|          | 0/3009 [00:00<?, ? examples/s]

Map:   0%|          | 0/2213 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'paragraphs', 'relevant', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 21714
    })
    validation: Dataset({
        features: ['id', 'question', 'paragraphs', 'relevant', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3009
    })
    test: Dataset({
        features: ['id', 'question', 'paragraphs', 'relevant', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2213
    })
})


In [8]:
from transformers import Trainer, TrainingArguments


training_args = TrainingArguments(
    output_dir="./results",
    # eval_strategy="epoch",
    # eval_dataset='Trainer',
    run_name="3B",
    learning_rate=2e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    # fp16=False,
    # bf16=True,
    group_by_length=True,
    report_to="wandb"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"]
)

trainer.train()


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.741800
1000,0.617000
1500,0.627100
2000,0.603800
2500,0.596000
3000,0.610300
3500,0.605500
4000,0.588400
4500,0.591400
5000,0.593300


TrainOutput(global_step=10857, training_loss=0.595125819390086, metrics={'train_runtime': 8844.0544, 'train_samples_per_second': 2.455, 'train_steps_per_second': 1.228, 'total_flos': 4.708292680640102e+16, 'train_loss': 0.595125819390086, 'epoch': 1.0})

In [9]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: fineGr

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_dir = "Dmgy/FineTunning-Llama3.2-3B-Instruct"

# push model to Hub
model.push_to_hub(model_dir, check_pr=True)
tokenizer.push_to_hub(model_dir)


adapter_model.safetensors:   0%|          | 0.00/18.4M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Dmgy/FineTunning-Llama3.2-3B-Instruct/commit/76f95ea99cba5679fcd439ec6a7ed0ee7716d2a3', commit_message='Upload tokenizer', commit_description='', oid='76f95ea99cba5679fcd439ec6a7ed0ee7716d2a3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Dmgy/FineTunning-Llama3.2-3B-Instruct', endpoint='https://huggingface.co', repo_type='model', repo_id='Dmgy/FineTunning-Llama3.2-3B-Instruct'), pr_revision=None, pr_num=None)